In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import seaborn as sns

from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers, metrics
from keras.callbacks import EarlyStopping

from Preprocessing.preprocessor_encoder import *


2023-08-30 09:30:30.109417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 09:30:30.222656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-30 09:30:30.222669: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-30 09:30:30.247130: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-30 09:30:30.879866: W tensorflow/stream_executor/platform/de

####
Model types

-rf
-sequential densew
-classification 

activation
-ReLu 
-Softmax


In [2]:
df = pd.read_csv("../raw_data/horse_racing_raw.csv")
df.head(3)

,id,f_id,f_ko,f_track,f_going,f_racetype,f_horse,f_jockey,f_trainer,f_distance,...,f_pm_05m,f_pm_03m,f_pm_02m,f_pm_01m,f_bsp_p_back,f_bsp_p_lay,f_pm_01m_p_back,f_pm_01m_p_lay,f_pm_15m_p_back,f_pm_15m_p_lay
0,16916847000316,16916847000002,2023-08-10 17:25:00,YARMOUTH,GD,Other Handicap,Porfin,Molly Presland,Phil McEntee,7.0,...,9.00,10.00,9.80,7.60,-1.00,0.95,-1.00,0.95,-1.00,0.95
1,16916953800243,16916953800013,2023-08-10 20:23:00,SALISBURY,GD,Other Handicap,Manxman,Robert Havlin,Simon & Ed Crisford,14.0,...,2.02,1.96,2.04,2.00,0.96,-1.01,0.95,-1.02,1.06,-1.14
2,16916829000067,16916829000014,2023-08-10 16:55:00,YARMOUTH,GD,Other Handicap,Bryce,Faye McManoman,Nigel Tinkler,6.0,...,4.60,4.70,4.40,3.35,-1.00,0.95,-1.00,0.95,-1.00,0.95


In [3]:
df_clean = preprocess_features(df.head(50000))


/home/conno/code/lucasglanville/and_theyre_off_backend/Preprocessing/preprocessor_encoder.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['pred_isp'] = X['pred_isp'].apply(odds_to_prob)
/home/conno/code/lucasglanville/and_theyre_off_backend/Preprocessing/preprocessor_encoder.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['f_pm_01m'] = X['f_pm_01m'].apply(odds_to_prob)
/home/conno/code/lucasglanville/and_theyre_off_backend/Preprocessing/preprocessor_encoder.py:24: SettingWithCopyWarning: 
A va

✅ ODDS CONVERTED TO PROBABILITY (1/ODDS)
✅ WINNERS CODED AS '1'
✅ 'f_ko' CONVERTED TO DATETIME
✅ TRACK CONDITIONS ORDINALLY ENCODED
✅ DROPPED ROWS WITH NULL VALUES
✅ NUMERIC FEATURES MINMAX-SCALED
✅ CAT. FEATURES OH-ENCODED (Track, Jockey, Trainer, Racetype)
✅ COLUMN TRANSFORMER ASSEMBLED
⏳ FIT_TRANSFORMING FEATURES...
✅ X_PROCESSED WITH SHAPE: (49977, 2942)


In [9]:
df_clean = df_clean.select_dtypes(exclude=['object'])

AttributeError: 'DataFrame' object has no attribute 'f_ko'

In [ ]:
val_start_date = '2022-02-20 18:45:00'
val_end_date = '2022-08-20 18:45:00'

train = df_clean[df_clean['f_ko'] <= val_start_date]
val = df_clean[(df_clean['f_ko'] > val_start_date) & (df_clean['f_ko'] <= val_end_date)]
test = df_clean[df_clean['f_ko'] > val_end_date]

In [ ]:
X_train = train.drop(['f_score', 'f_ko'])
y_train = train.f_score

X_val = val.drop(['f_score', 'f_ko'])
y_val = val.f_score

X_test = test.drop(['f_score', 'f_ko'])
y_test = test.f_score

In [ ]:
X_train.info()

In [ ]:
def initialize_model():

  model = Sequential()

  model.add(layers.Dense(100, input_dim=X_train.shape[-1], activation='relu'))
  model.add(layers.Dense(60, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(25, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  ###Compile###
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model


model = initialize_model()
model.summary()

In [ ]:
es = EarlyStopping(
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

history = model.fit(
        X_train,
        y_train,
        epochs=500,
        batch_size=32,
        validation_data=(X_val,y_val),
        callbacks=[es],
        verbose=1
    )

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(y_pred_bool)

print(classification_report(y_test, y_pred_bool))


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [ ]:
len(y_pred[:,0])

In [ ]:
X_test[-2:]

In [ ]:
pd.DataFrame([y_test.reset_index(drop=True), y_pred[:,0]])

In [ ]:
plt.scatter(pd.Series(y_pred[:,0]).value_counts(),pd.Series(y_pred[:,0]).value_counts().index )

In [ ]:
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred_bool